#An illustrated love poem generator using GPT and DALL-E (OpenAI)



## Upgrading tools and installing OpenAI

In [4]:
!pip install --upgrade pip
!pip install setuptools wheel
!pip install openai



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 26.1 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 22.0.4
    Uninstalling pip-22.0.4:
      Successfully uninstalled pip-22.0.4
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.3/57.3 kB 2.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 kB 28.5

In [51]:
import openai 
import os
import requests
import shutil

from PIL import Image, ImageFont, ImageDraw 

### The API key should stay private

In [52]:
#os.environ["OPENAI_API_KEY"] = 'PUT_API_KEY_HERE'

In [7]:
openai.api_key = os.getenv('OPENAI_API_KEY')

# Making a GPT query

In [8]:
response = openai.Completion.create(
    model='text-davinci-003',
    prompt='Give me two reasons to learn OpenAI API with Python', 
    max_tokens=300)

In [9]:
print(response['choices'][0]['text'])



1) OpenAI API with Python is a powerful tool for developing and deploying AI applications. This programming language provides a set of robust, easy-to-use tools and libraries that can be used to quickly create and deploy AI projects.

2) OpenAI API with Python is becoming increasingly popular as a tool for machine learning, natural language processing, computer vision, and reinforcement learning. Its flexible, Python-based libraries, APIs, and tools make it suitable for a wide range of AI-related tasks, such as object recognition, text analysis, decision-making, and robotics.


### OpenAI rejects unethical queries, and an account can be paused if too many queries are flagged. To avoid that we can test if a query is acceptable:

In [10]:
response = openai.Moderation.create(input="is this an acceptable query ?")
response["results"][0]

<OpenAIObject at 0x7f1750242d10> JSON: {
  "categories": {
    "hate": false,
    "hate/threatening": false,
    "self-harm": false,
    "sexual": false,
    "sexual/minors": false,
    "violence": false,
    "violence/graphic": false
  },
  "category_scores": {
    "hate": 6.822017439844785e-06,
    "hate/threatening": 1.954006574600431e-11,
    "self-harm": 5.504936506639524e-10,
    "sexual": 4.826387521461584e-05,
    "sexual/minors": 4.548151537164813e-06,
    "violence": 1.1513262343498809e-08,
    "violence/graphic": 4.424981625561486e-08
  },
  "flagged": false
}

# Asking GPT to create a short poem

In [17]:
def create_poem_prompt(name, author, lines = 8):
  prompt = f"A {lines} lines poem that rimes, about a woman named {name}, in the style of {author}:"
  return prompt

def generate_GPT_poem():
  response = openai.Completion.create(
    model='text-davinci-003',
    prompt=create_poem_prompt("Anouk","Pablo Neruda"), 
    temperature = 0.3,
    max_tokens=300)
  return response['choices'][0]['text']


In [53]:
poem = generate_GPT_poem()
print(poem)



Anouk, a woman of beauty and grace,
A heart of gold, a smile so sweet to face.
Her eyes so bright, her hair so fair,
A gentle soul, a beauty so rare.

Her spirit so strong, her will so sure,
Her courage and strength, an inspiration pure.
A kind and gentle woman, a heart so true,
Anouk, a woman of beauty, so lovely to view.


# Generating a nice picture with DALL-E

In [54]:
def generate_DALLE_image_url():
  response = openai.Image.create(
    prompt="a romantic digital art painting of a couple of lovers",
    n=1,
    size="1024x1024")
  return response['data'][0]['url']

def save_image(image_url, filename):
  image_res = requests.get(image_url, stream=True)
  if image_res.status_code == 200:
    with open(filename, 'wb') as f:
      shutil.copyfileobj(image_res.raw, f)
  else:
    print ('There was an error while loading the image')
  return image_res.status_code

In [55]:
image_url = generate_DALLE_image_url()


In [56]:
save_image(image_url, 'poem_image.png')

200

# Combining image and text with Pillow

In [57]:

image = Image.open('poem_image.png')
draw = ImageDraw.Draw(image)
draw.font = ImageFont.truetype("DancingScript-Regular.ttf", 50)
x = 100
y = 50
# shadow
draw.multiline_text((x+2,y+2), poem, fill=(50, 50, 50), align='left')
# text
draw.multiline_text((x,y), poem, fill=(255, 128, 128), align='left')
image.save('poem.png')